# 11 · Filtro de la **Media** — Interactivo (layout limpio, sin duplicados)

**Layout**
1) Sliders arriba.  
2) Debajo: **imagen ruidosa** (izq) y **imagen filtrada** (der).  
3) Debajo: **resumen de parámetros** (texto).  
4) Debajo: **matriz del kernel** (numérica) y su suma.

**Notas**
- `BORDER_WRAP` no es compatible con `blur/boxFilter`;
- En `boxFilter` con `normalize=False` la salida real es **suma** 


In [1]:
import cv2, numpy as np, matplotlib.pyplot as plt
try:
    import ipywidgets as widgets
    from ipywidgets import HBox, VBox, interactive_output
    _W_OK = True
except Exception:
    print("⚠️ ipywidgets no está instalado. Instala con `pip install ipywidgets`.")
    _W_OK = False

# --- Generadores de ruido ---
def add_gaussian_noise(img, mean=0.0, sigma=25.0):
    noise = np.random.normal(mean, sigma, img.shape).astype(np.float32)
    noisy = img.astype(np.float32) + noise
    return np.clip(noisy, 0, 255).astype(np.uint8)

def add_salt_pepper(img, prob=0.02):
    noisy = img.copy()
    h, w = img.shape[:2]
    mask = np.random.rand(h, w)
    for c in range(img.shape[2]):
        ch = noisy[..., c]
        ch[mask < prob/2] = 0
        ch[mask > 1 - prob/2] = 255
        noisy[..., c] = ch
    return noisy

# --- Bordes ---
BORDER_MAP = {
    "BORDER_DEFAULT": cv2.BORDER_DEFAULT,
    "BORDER_CONSTANT": cv2.BORDER_CONSTANT,
    "BORDER_REPLICATE": cv2.BORDER_REPLICATE,
    "BORDER_REFLECT": cv2.BORDER_REFLECT,
    "BORDER_REFLECT_101": cv2.BORDER_REFLECT_101,
    "BORDER_WRAP": cv2.BORDER_WRAP,
}

# --- Imagen ---
IMG_PATH = r"C:\Users\20808\Documents\Repositorios\Vision_en_Robotica\imagenes\lenna.png"
img_bgr = cv2.imread(IMG_PATH)
if img_bgr is None:
    raise FileNotFoundError("No se encontró la imagen. Ajusta IMG_PATH.")
img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)

def render(noise_type='Gaussiano', sigma=25, prob=0.02,
           kx=5, ky=5, method='blur', border='BORDER_REFLECT'):
    # Una sola salida por llamada
    kx = max(1, int(kx)); ky = max(1, int(ky))

    # Imagen ruidosa
    if noise_type == 'Gaussiano':
        noisy = add_gaussian_noise(img_rgb, sigma=sigma)
    else:
        noisy = add_salt_pepper(img_rgb, prob=prob)

    # Borde (fallback para WRAP)
    border_cv = BORDER_MAP.get(border, cv2.BORDER_REFLECT)
    effective_border = border
    if border_cv == cv2.BORDER_WRAP:
        border_cv = cv2.BORDER_REFLECT
        effective_border = "BORDER_REFLECT  (fallback de WRAP)"

    # Filtro y kernel
    if method == 'blur':
        filtered = cv2.blur(noisy, (kx, ky), borderType=border_cv)
        K = np.ones((ky, kx), np.float32) / float(kx*ky)
        kernel_info = f"Suma kernel = {K.sum():.3f} (media normalizada)"
        extra = ""
    elif method == 'boxFilter norm':
        filtered = cv2.boxFilter(noisy, ddepth=-1, ksize=(kx, ky),
                                 normalize=True, borderType=border_cv)
        K = np.ones((ky, kx), np.float32) / float(kx*ky)
        kernel_info = f"Suma kernel = {K.sum():.3f} (media normalizada)"
        extra = ""
    else:  # 'boxFilter suma'
        filt32 = cv2.boxFilter(noisy.astype(np.float32), ddepth=cv2.CV_32F,
                               ksize=(kx, ky), normalize=False, borderType=border_cv)
        filtered = np.clip(filt32 / float(kx*ky), 0, 255).astype(np.uint8)
        K = np.ones((ky, kx), np.float32)
        kernel_info = f"Suma kernel = {K.sum():.0f} (suma); mostrado normalizado para visualizar"
        extra = (f"Rango salida (suma, por canal): "
                 f"R=({filt32[...,0].min():.1f},{filt32[...,0].max():.1f})  "
                 f"G=({filt32[...,1].min():.1f},{filt32[...,1].max():.1f})  "
                 f"B=({filt32[...,2].min():.1f},{filt32[...,2].max():.1f})")

    # --- Imágenes ---
    plt.figure(figsize=(12,5))
    plt.subplot(1,2,1); plt.imshow(noisy);    plt.title(f"Ruidosa — {noise_type}"); plt.axis('off')
    plt.subplot(1,2,2); plt.imshow(filtered); plt.title(f"Filtrada — {method}");   plt.axis('off')
    plt.tight_layout(); plt.show()

    # --- Resumen de parámetros ---
    text = f"""Parámetros:
- Ruido: {noise_type} (σ={sigma}, p={prob})
- Filtro: {method}
- ksize=({kx},{ky})
- borderType: {effective_border}
"""
    if extra:
        text += f"\n{extra}"
    print(text)

    # --- Kernel numérico ---
    print(kernel_info)
    np.set_printoptions(precision=4, suppress=True)
    print(K)

if _W_OK:
    noise_dd  = widgets.Dropdown(options=['Gaussiano','Sal&Pimienta'], value='Gaussiano', description='Ruido:')
    sigma_sl  = widgets.IntSlider(value=25, min=0, max=60, step=1, description='σ (Gauss):')
    prob_sl   = widgets.FloatSlider(value=0.02, min=0.0, max=0.1, step=0.005, readout_format='.3f', description='p (S&P):')
    kx_sl     = widgets.IntSlider(value=5, min=1, max=31, step=1, description='kx')
    ky_sl     = widgets.IntSlider(value=5, min=1, max=31, step=1, description='ky')
    method_dd = widgets.Dropdown(options=['blur','boxFilter norm','boxFilter suma'], value='blur', description='Método:')
    border_dd = widgets.Dropdown(options=list(BORDER_MAP.keys()), value='BORDER_REFLECT', description='Borde:')

    ui = VBox([
        HBox([noise_dd, sigma_sl, prob_sl]),
        HBox([kx_sl, ky_sl]),
        HBox([method_dd, border_dd])
    ])

    out = interactive_output(
        render,
        {
            'noise_type': noise_dd,
            'sigma': sigma_sl,
            'prob': prob_sl,
            'kx': kx_sl,
            'ky': ky_sl,
            'method': method_dd,
            'border': border_dd
        }
    )

    display(ui, out)
else:
    print("Interactividad deshabilitada (instala ipywidgets).")

Output()